In [1]:
import os

openai_key = os.getenv("OPENAI_API_KEY")
openai_base = os.getenv("OPENAI_BASE")
openai_deployment = os.getenv("AZURE_OPEN_AI_MODEL")


os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://prod-open-ai-service.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "30c82b416ff4456faa26b1644a83ab4b"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [2]:
DATA_PATH = "../data-legal"

from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(DATA_PATH).load_data()

In [4]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    StorageContext,
    SimpleDirectoryReader,
)
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import LangChainLLM
from llama_index import LangchainEmbedding

# load documents


# initialize service context (set chunk size)
# -- here, we set a smaller chunk size, to allow for more effective re-ranking
llm = AzureOpenAI(
    engine="PROD-GPT-16K-TURBO",
    temperature=0, openai_api_type="azure",
    openai_api_key=openai_key,
    openai_api_base=openai_base,
    openai_api_version="2023-05-15")

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        deployment="PROD-text-embedding-ada-002",
        openai_api_type="azure",
        openai_api_key=openai_key,
        openai_api_base=openai_base,
        openai_api_version="2023-05-15",
        chunk_size=1500,
    ),
    embed_batch_size=16,
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)
# service_context = ServiceContext.from_defaults(llm=llm)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(documents)

In [5]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [31]:
from llama_index.retrievers import BM25Retriever

# retireve the top 10 most similar nodes using embeddings
vector_retriever = index.as_retriever(similarity_top_k=30)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(index, similarity_top_k=)

In [32]:
from llama_index.retrievers import BaseRetriever


class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

In [33]:
index.as_retriever(similarity_top_k=30)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

## Reranker

In [22]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(
    top_n=30, model="cross-encoder/ms-marco-MiniLM-L-6-v2"
)

In [ ]:
from llama_index.indices.postprocessor import LLMRerank

reranker = LLMRerank(service_context=service_context)



In [37]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker],
    service_context=service_context,
)

response = query_engine.query("What is the name of the borrower ?")

IndexError: list index out of range

In [ ]:
from llama_index.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** The name of the borrower in the loan agreement is Borrower.